In [173]:
# Importing the Libraries
import pandas as pd
from pandas.util import hash_pandas_object
import math
from datetime import datetime
import pathlib
import csv
import json
import random
import string
import hashlib

# Data Cleaning

In [174]:

# Reading the data file
df = pd.read_csv('Transaction.csv', sep=r'\s*,\s*',
                            encoding='ascii', engine='python')

df_imputed = df.iloc[:, 0:17] # Dropping Nan column 17, 18 as it has got NaN values 
df_sorted_by_response_time = df.iloc[:, 0:17]
df_imputed = df_imputed.fillna(1) # Filling the null value with 1
nan_values = df_imputed.isnull().sum()
display(nan_values) # Displaying missing or NaN values after cleaning the data


Account_ID           0
CODE                 0
Implemented Date     0
Active Indicator     0
Account Type         0
Service              0
BU                   0
Request Date         0
Account status       0
Status Code          0
$ Amount             0
Version              0
Agent ID             0
FIBRE                0
last Updated Date    0
Property TYPE        0
Post Code            0
dtype: int64

In [175]:
# Change the Account_ID Data Type in order to do hashing 
df_imputed.Account_ID = df_imputed.Account_ID.astype(str)

d0_initial = df_imputed # make a copy of the cleaned data bfore hashing

# Create a list of the data from the sensitive, clear text column (‘Account_ID’ in this case)
data = d0_initial[['Account_ID','CODE','Implemented Date', 'Account Type', 'Service', 'BU','Request Date', 'Account status', '$ Amount',
         'Agent ID', 'last Updated Date', 'Property TYPE', 'Post Code']]
display(data.head())
data.shape

,Account_ID,CODE,Implemented Date,Account Type,Service,BU,Request Date,Account status,$ Amount,Agent ID,last Updated Date,Property TYPE,Post Code
0,15368.0,543,7/09/2017 0:00,B2B,ELE,NUOS,17/08/2017 0:00,PAID,3.94,307508,7/09/2017 11:09,RESI,2587
1,23232.0,543,7/09/2017 0:00,B2B,ELE,NUOS,10/08/2017 0:00,PAID,126.96,307508,7/09/2017 11:04,RESI,2587
2,232323.0,543,7/09/2017 0:00,B2B,ELE,NUOS,9/08/2017 0:00,PAID,15.43,307508,7/09/2017 11:05,RESI,2587
3,307263.0,543,7/09/2017 0:00,B2B,ELE,NUOS,9/08/2017 0:00,PAID,3.49,307508,7/09/2017 11:05,RESI,2587
4,415740.0,105,5/09/2017 0:00,B2B,ELE,NUOS,4/09/2017 0:00,PAID,3.31,307312,11/09/2017 10:44,RESI,2581


(30320, 13)

# 1. Augment the data with a hash key

In [176]:

# Get a unique list of the clear text, as a List
tmplist = list(set(data['Account_ID']))

# Add some random characters before and after the Account_ID.
mapping1 = {i : (''.join(random.choice(string.hexdigits) for i in range(12)))+i+(''.join(random.choice(string.hexdigits) for i in range(12)))  for i in tmplist}


# Create a new DF (as d2 in the code, just to leave the original d1 intact) and create a new hash column.

d2_frame = data.copy()# Create a new column containing clear_text_Nonce
d2_frame['Hash_Account_ID'] = [mapping1[i] for i in d2_frame['Account_ID']]

# Generate hashes in hexadecimals (0–9, a-f, and A-F)
d2_frame['hash'] = [hashlib.sha1(str.encode(str(i))).hexdigest() for i in d2_frame['Hash_Account_ID']]


# Create a new DF (data_final) with the clear-text column removed.
data_final = d2_frame[['hash','CODE','Implemented Date', 'Account Type', 'Service', 'BU', 'Request Date', 'Account status', '$ Amount',
        'Agent ID', 'last Updated Date', 'Property TYPE', 'Post Code']].rename(columns={'hash':'Hash_Account_ID'})
display(data_final.head()) 

# Saving Hash file into the working directory
data_final.to_csv(r'data_hashed.csv', index = False)

,Hash_Account_ID,CODE,Implemented Date,Account Type,Service,BU,Request Date,Account status,$ Amount,Agent ID,last Updated Date,Property TYPE,Post Code
0,6dd789732991b823fbf43cb9362383e98c979988,543,7/09/2017 0:00,B2B,ELE,NUOS,17/08/2017 0:00,PAID,3.94,307508,7/09/2017 11:09,RESI,2587
1,15a285fd7ce7caf1caa06a8be7b1faf70bcc4ec6,543,7/09/2017 0:00,B2B,ELE,NUOS,10/08/2017 0:00,PAID,126.96,307508,7/09/2017 11:04,RESI,2587
2,86bb72793e4c7e32bf017a5f6738317001e836e2,543,7/09/2017 0:00,B2B,ELE,NUOS,9/08/2017 0:00,PAID,15.43,307508,7/09/2017 11:05,RESI,2587
3,8f5116b83268ff39ff803033513dd183b01961f6,543,7/09/2017 0:00,B2B,ELE,NUOS,9/08/2017 0:00,PAID,3.49,307508,7/09/2017 11:05,RESI,2587
4,f80fc31d50921402700f9e82f201a272680ee7b3,105,5/09/2017 0:00,B2B,ELE,NUOS,4/09/2017 0:00,PAID,3.31,307312,11/09/2017 10:44,RESI,2581


# 2. Filter out the questionable data (Negative Amount)

In [177]:
# Sort the data by negative amount
df_questionable = data_final.sort_values(by='$ Amount')

display(df_questionable)



,Hash_Account_ID,CODE,Implemented Date,Account Type,Service,BU,Request Date,Account status,$ Amount,Agent ID,last Updated Date,Property TYPE,Post Code
4189,2d0af69207887960157d8f3f8be8592bbda8f58e,543,7/09/2017 0:00,B2B,ELE,NUOS,28/08/2017 0:00,PAID,-7088.09,307511,7/09/2017 10:59,1,2081
1007,1dd90cbffc75c1bb0a5de479dcfaa5e9e94e429e,1124,7/09/2017 0:00,B2B,ELE,NUOS,23/08/2017 0:00,PAID,-3137.64,307507,7/09/2017 10:58,1,2081
6881,22992461f7201f13d7618b679e6ed8ade533168c,543,7/09/2017 0:00,B2B,ELE,NUOS,10/08/2017 0:00,PAID,-3079.55,307511,7/09/2017 10:59,BEST,2081
2015,87cf9513681d45d619b4b6f633d1340a3f111f11,543,7/09/2017 0:00,B2B,ELE,NUOS,10/08/2017 0:00,PAID,-1291.01,307511,7/09/2017 10:59,1,2081
5213,24a43de76497965bfd756a78692930cc02f53682,543,7/09/2017 0:00,B2B,ELE,NUOS,10/08/2017 0:00,PAID,-1254.99,307511,7/09/2017 10:59,1,2081
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2312,131ed6c8e1fa9d061ae9f85bc3a19df9bbf641fd,543,7/09/2017 0:00,B2B,ELE,NUOS,4/09/2017 0:00,PAID,3800.50,307511,7/09/2017 10:59,1,2081
6087,c1379e38d0b0595f4fbba83f15255176b4918e92,1124,7/09/2017 0:00,B2B,ELE,NUOS,4/09/2017 0:00,PAID,3946.80,307507,7/09/2017 10:58,1,2081
3867,a7b41001a7782e8ca31790d1841bed50f72611d2,1124,7/09/2017 0:00,B2B,ELE,NUOS,1/09/2017 0:00,PAID,4216.00,307507,7/09/2017 10:58,BEST,2081
7154,232b0494757cc55975c3f0f9b7cf0a8f821a9c69,543,7/09/2017 0:00,B2B,ELE,NUOS,1/09/2017 0:00,PAID,4355.92,307511,7/09/2017 10:59,1,2081


# 3. Write the data in a JSON file one record at a time
   4. Watch the data emitted in the previous task
   5. When the number of events reach 1000, output the events to a JSON file

The output Jason files will be stored in the working directory 

In [178]:
JSON_ENTRIES_THRESHOLD = 1000  # Number of Events 1000

# Write to Jason File
def write_json(json_array, filename):
    with open(filename, 'w', encoding='utf-8') as jsonf:
        
        jsonString = json.dumps(json_array, indent=4)
        jsonf.write(jsonString)

def csv_to_json(csvFilePath):
    jsonArray = []

    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
        filename_index = 0

        for row in csvReader:
            jsonArray.append(row)
            # display(jsonArray) #this will show each and every line as it is recording. It takes too long to complete
            if len(jsonArray) >= JSON_ENTRIES_THRESHOLD:
                #display(jsonArray)# Show jason recording after 1000 lines 
                # if we reached the treshold, write out
                write_json(jsonArray, f"{filename_index}.json")
                filename_index += 1
                jsonArray = []

        # Finally, write out the remainder
        write_json(jsonArray, f"{filename_index}.json")


csvFilePath = r'data_hashed.csv' # csv file location previously saved when done hashing
csv_to_json(csvFilePath)

# 6. List post codes based on fastest response (Sort by Response Time)

In [179]:
# Convert Implemented Date and Request Date columns to datetime format and deduct the number of days 
datetimeformat = '%d/%m/%Y %H:%M'
implemented_date_series =  pd.to_datetime(data_final.iloc[:, 2], format=datetimeformat)
request_date_series =  pd.to_datetime(data_final.iloc[:, 6], format=datetimeformat)

response_time = implemented_date_series - request_date_series
df_sorted_by_response_time["Response Time "] = response_time
df_sorted_by_response_time = df_sorted_by_response_time.sort_values('Response Time ')

# Saving  List post codes based on fastest response file into the working directory
df_sorted_by_response_time.to_csv(r'fastest_response.csv', index = False)
df_sorted_by_response_time


,Account_ID,CODE,Implemented Date,Active Indicator,Account Type,Service,BU,Request Date,Account status,Status Code,$ Amount,Version,Agent ID,FIBRE,last Updated Date,Property TYPE,Post Code,Response Time
19806,10001.0,108,8/09/2017 0:00,1,B2B,ELE,NUOS,7/09/2017 0:00,PAID,5,7.30,v1_0,307562,EN63321345-N-2,13/09/2017 9:02,RESI,2581,1 days
20578,10073.0,543,7/09/2017 0:00,1,B2B,ELE,NUOS,6/09/2017 0:00,PAID,9,4.29,v1_0,307510,EN122672134-N-8,7/09/2017 13:17,RESI,2587,1 days
5913,10094.0,64,7/09/2017 0:00,1,B2B,GAS,DUOSV,6/09/2017 0:00,PAID,3,61.39,v1_0,307487,2.67E+14,3/10/2017 12:57,RESI,2584,1 days
5912,10097.0,64,7/09/2017 0:00,1,B2B,GAS,DUOSV,6/09/2017 0:00,PAID,7,117.07,v1_0,307487,2.67E+14,3/10/2017 12:57,RESI,2584,1 days
5911,10044.0,64,7/09/2017 0:00,1,B2B,GAS,DUOSV,6/09/2017 0:00,PAID,6,74.49,v1_0,307487,2.67E+14,3/10/2017 12:58,RESI,2584,1 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28420,10018.0,107,11/09/2017 0:00,1,B2B,ELE,NUOS,8/09/2016 0:00,PAID,8,-1.75,v1_0,307664,CEN99573813-C-8,18/09/2017 13:13,RESI,2582,368 days
22062,10047.0,107,11/09/2017 0:00,1,B2B,ELE,NUOS,29/08/2016 0:00,PAID,4,-38.49,v1_0,307664,CEN98648458-C-8,18/09/2017 13:13,RESI,2582,378 days
19948,10039.0,107,11/09/2017 0:00,1,B2B,ELE,NUOS,14/07/2016 0:00,PAID,0,-0.59,v1_0,307662,CEN94977071-C-2,18/09/2017 13:14,RESI,2582,424 days
20513,10037.0,107,11/09/2017 0:00,1,B2B,ELE,NUOS,26/11/2014 0:00,PAID,7,-10.46,v1_0,307662,CEN51544029-C-2,18/09/2017 13:14,RESI,2582,1020 days
